# Tableau Rest API 
## Analytics Day 2025



Thanks for attending my presentation "A Dashboard for My Dashboards." I hope this script can help you get started working with the Tableau REST API. For more information, I recommend the following resources: 

- `tableau_api_lib` documentation: https://pypi.org/project/tableau-api-lib/
- Tableau API versions: https://help.tableau.com/current/api/rest_api/en-us/REST/rest_api_concepts_versions.htm
- Response Codes: https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status

### Package Requirements

`pip install pandas -U`

`pip install tableau_api_lib -U`

`pip install getpass -U`

In [ ]:
from tableau_api_lib import TableauServerConnection # this will create a connection string and connect to Tableau Server
from tableau_api_lib.utils import querying # this will help us get data the data we want from Tableau and put it in a Pandas dataframe

import pandas as pd # pandas will help with any data transformation operations
import getpass

### Connection String

Configuring the connection string will point your computer to your specific Tableau server and site. You'll need a few things to create this. 
1. **Server Name and Site Name**: This is the server and site where you keep all your dashboards. To find your server and site name, look at the url in your browser. It should look like `https://tableau.my-institution.edu/#/site/my-site/`. In this case **`https://tableau.my-institution.edu/`** is the server name and **`my-site`** is the site name. 


In [ ]:
server_name = '' # type your server name here
site_name = '' # type your site name here
site_url = '' # type your site url here. 

2. **Token**: Before running the next chunck of code, go to your **Account Settings** and scroll down to **Personal Access Tokens**. Write your token name, click **Create Token** and copy the secret Tableau generates for you. 

In [ ]:
# Running this code will open two text boxes. Type in your Token Name and Secret here.
token_name = getpass.getpass(prompt='Token Name: ', stream=None)
secret = getpass.getpass(prompt='Secret: ' , stream=None)

3. **API Version**: Tableau Cloud and Server require different API versions. Check the table below to see which version you need. For more information on API versions, check https://help.tableau.com/current/api/rest_api/en-us/REST/rest_api_concepts_versions.htm#rest_api_versioning
|Tableau | Cloud/Server version |	REST API version |	Schema version|
|---|---|---|---|
|2025.2| (Tableau Cloud only) |	3.26 	|3.26|
|2025.1| 	|3.25 	|3.25|
|2024.3| (Tableau Cloud only)| 	3.24| 	3.24|
|2024.2| 	|3.23 	|3.23|
|2024.1| (Tableau Cloud only) |	3.22 |	3.22|
|2023.3 |	|3.21| 	3.21|
|2023.2| (Tableau Cloud only)| 	3.20 |	3.20|
|2023.1| 	|3.19| 	3.19|
|2022.4| (Tableau Cloud only)| 	3.18| 	3.18|
|2022.3| 	|3.17| 	3.17|
|2022.2| (Tableau Cloud only) |	3.16 |	3.16|

In [ ]:
# Check API version and get site name
api_version = '' # type your API version between the single quotation marks

Run the code below to log in. Response 200 means you've logged-in successfully. Responses in the 400-500 range means there may be something wrong. Double-check your server and account details. For more information about response codes, see: https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status

In [ ]:
# create connection
config = {
    "My Tableau Server": {  # Name this anything you want! If you're connecting to dev and production servers, you can distinguish between them here
        "server": server_name,
        "api_version": api_version,
        "personal_access_token_name": token_name,
        "personal_access_token_secret": secret,
        "site_name": site_name,
        "site_url": site_url
    }
}

c = TableauServerConnection(config, env='My Tableau Server')
c.sign_in()

### Query Your Server

The code below will bring in data about your Views, Workbooks and Projects. Use check Tableau API Lib documents to see what else you can find!

In [ ]:
view = querying.get_views_dataframe(conn)

print('Number of Views:', len(view))
view.head() # get output for first 5 rows

In [ ]:
# Clean View DataFrame
view['workbook'] = pd.json_normalize(view['workbook'])['id']
view['owner'] = pd.json_normalize(view['owner'])['fullName']
view['project'] = pd.json_normalize(view['project'])['id']
view = view[['workbook','tags','owner','project','id','name','contentUrl','createdAt','updatedAt','sheetType','viewUrlName','usage_totalViewCount']]

In [ ]:
workbook = pd.DataFrame(extract_pages(conn.query_workbooks_for_site))

print('Number of Workbooks:', len(workbook))
workbook.head()

In [ ]:
# Clean Workbook DataFrame
workbook['owner'] = pd.json_normalize(workbook['owner'])['fullName']
workbook['project'] = pd.json_normalize(workbook['project'])['id']
workbook = workbook[['id','project','name','description','owner','contentUrl','webpageUrl','size','createdAt','updatedAt','defaultViewId']]

In [ ]:
project = pd.DataFrame(extract_pages(conn.query_projects))

print('Number of Projects:', len(project))
project.head()

In [ ]:
# Clean Project DataFrame

project['owner'] = pd.json_normalize(project['owner'])['fullName']
project = project[['id','name','description','owner','createdAt','updatedAt','contentPermissions','parentProjectId','controllingPermissionsProjectId']]
project = pd.merge(project,project[['id','name']], left_on='parentProjectId',right_on='id', how='left').drop(columns='id_y').rename(columns={'id_x':'id','name_x':'project_name','name_y':'parentProjectName'})

### Save to CSV files

In [ ]:
path = '' #Define your file path. In Python, this should be formatted "C:/User/Documents/Folder/"

project.to_csv(path+'tableau_project.csv')

In [ ]:
# Sign out and close connection
conn.sign_out() # <Response [204]> means success